In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('complete_narratives.csv')
df

,Unnamed: 0,doc,sentence,statement,ARG0_highdim,ARG0_lowdim,B-V_highdim,B-V_lowdim,B-ARGM-NEG_highdim,B-ARGM-NEG_lowdim,B-ARGM-MOD_highdim,ARG1_highdim,ARG1_lowdim,ARG2_highdim,ARG2_lowdim,B-V_lowdim_with_neg,B-V_highdim_with_neg,narrative_highdim,narrative_lowdim
0,1,98454970654916608,0,1,republican democrat,republican,create,create,NaN,NaN,NaN,economic problem,economy,NaN,NaN,not_create,not_create,republican democrat not_create economic problem,republican not_create economy
1,6,1234653427789070336,1,6,hardworking american patriot,american,love,know,NaN,NaN,NaN,country,country,NaN,NaN,not_know,not_love,hardworking american patriot not_love country,american not_love country
2,7,1234653427789070336,1,7,hardworking american patriot,american,cherish,cherish,NaN,NaN,NaN,value,million,NaN,NaN,not_cherish,not_cherish,hardworking american patriot not_cherish value,american not_cherish million
3,8,1234653427789070336,1,8,hardworking american patriot,american,respect,honor,NaN,NaN,NaN,law,law,NaN,NaN,not_honor,not_respect,hardworking american patriot not_respect law,american not_respect law
4,9,1234653427789070336,1,9,hardworking american patriot,american,NaN,NaN,NaN,NaN,NaN,america,america,NaN,NaN,not_,not_,hardworking american patriot not_ america,american not_ america
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15895,150187,1213149807948312576,68604,150187,career staff,team,NaN,NaN,NaN,NaN,NaN,million excuse possibly stop spending taxpayer...,money,NaN,NaN,not_,not_,career staff not_ million excuse possibly stop...,team not_ money
15896,150193,1213078681750573056,68606,150193,iran,iran,won,lose,True,NaN,NaN,war,dead,NaN,NaN,not_lose,won,iran won war,iran won dead
15897,150194,1213078681750573056,68606,150194,iran,iran,lose,win,True,NaN,NaN,negotiation,deal,NaN,NaN,not_win,lose,iran lose negotiation,iran lose deal
15898,150209,1319384118849949702,68612,150209,obama biden,biden,fail,fail,NaN,NaN,NaN,worker delphi,company,NaN,NaN,not_fail,not_fail,obama biden not_fail worker delphi,biden not_fail company


In [1]:
import re
import requests
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('opentapioca')

In [2]:
text = "Republicans and Democrats have both created our economic problems. Donald Trump didn't."
text = nlp(text)
text

Republicans and Democrats have both created our economic problems. Donald Trump didn't.

In [3]:
URL = 'https://query.wikidata.org/sparql'
    
QUERY = """
SELECT ?p_iri 
       ( SAMPLE(?p_label) AS ?p_label )
       ( GROUP_CONCAT( DISTINCT ?o ; SEPARATOR = "|" ) AS ?o_list )

WHERE {{
  {{
  
    SELECT ?p_iri   ?o_iri
           ?p_label ?o_label
           ( IF( ISIRI(?o_iri), CONCAT( ?o_label, "\\\\", STR(?o_iri) ), ?o_label ) AS ?o )
                  
    WHERE {{
    
      {} ?p ?o_iri .
      FILTER( !STRSTARTS( STR(?o_iri), "http://www.wikidata.org/entity/statement/" ) )
      ?p_iri wikibase:claim|wikibase:directClaim ?p .

      SERVICE wikibase:label {{
          bd:serviceParam wikibase:language "en" .
          ?p_iri rdfs:label ?p_label .
          ?o_iri rdfs:label ?o_label .
      }}
      
    }}
    
  }}
}}

GROUP BY ?p_iri
"""

In [4]:
class WdObject:
    """ Wikidata object """
    
    def __init__(self, label: str, iri: str = ""):
        self.label = label
        
        if iri:
            self.is_attr = False
            self.iri = self.format_iri(iri)
            self.has_prefix = iri != self.iri
        else:
            self.is_attr = True
            self.iri = ""
            self.has_prefix = False
            
    @staticmethod
    def format_iri(iri: str) -> str:
        """ Format IRI with appropriate prefix (if known) """
        return iri.replace("http://www.wikidata.org/entity/", "wd:")
        
    def __repr__(self) -> str:
        return self.label
    def __str__(self) -> str:
        return self.label
    
    def __hash__(self) -> str:
        return hash(self.iri if self.iri else self.label)

In [5]:
class NamedEntity:
    """ Named entity extracted from text, enriched with Wikidata information """
    
    def __init__(self, ent):
        self.label = ent.text
        self.type = ent.label_
        self.desc = ent._.description
        
        self.wid = ent.kb_id_
        self.iri = "wd:" + self.wid
        self.wd_ent = WdObject(self.label, self.iri)
        
        self.query = QUERY.format(self.iri)
        self.attrs, self.rels = {}, {}
        self.links = {}
        
    def __repr__(self) -> str:
        return self.label
    def __str__(self) -> str:
        return self.label
                
                
    def clean_attrs(self) -> None:
        """ Clean fetched attributes """
        to_del = []
        
        for p, o_list in self.attrs.items():
            
            if (
                # If attr is an ID
                re.findall("ID( *\(.*\))? *$", p.label) or
                p.label.isupper() or
                # TODO: To improve
                # If attr is a mixture of letters and numbers (likely not informative)
                (
                    o_list[0].label[0].isalpha() and
                    any(( not all(( char.isalpha() for char in o.label )) for o in o_list ))
                ) or
                (
                    o_list[0].label[0].isnumeric() and
                    any(( not all(( char.isnumeric() for char in o.label )) for o in o_list ))
                )
            ):
                to_del.append(p)
                continue
                
        for p in to_del:
            del self.attrs[p]
        
    
    def query_ent(self) -> None:
        """ Query WikiData knowledge base to enrich entity """
        
        params = { 'format': 'json', 'query': self.query }
        res = requests.get(URL, params=params)
        res = res.json()['results']['bindings']
        
        for r in res:
            
            p = WdObject(r['p_label']['value'], r['p_iri']['value'])
            o_list = r['o_list']['value'].split('|')
            
            rels, attrs = [], []
            for o in o_list:
                o = WdObject(*o.split("\\"))
                if o.is_attr:
                    attrs.append(o)
                else:
                    rels.append(o)
                    
            if rels:
                self.rels[p] = rels
            elif attrs:
                self.attrs[p] = attrs
                
        self.clean_attrs()
                
                
    def link_ents(self, ents: dict) -> None:
        """ Link to other named entities """
        for p, o_list in self.rels.items():
            for o in o_list:
                if o.iri in ents:
                    self.links[p] = o
                    
    def stringify_ents(self) -> None:
        """ Stringify relations to other entities that are not found elsewhere """
        for p, o_list in self.rels.items():
            if p not in self.links:
                for o in o_list:
                    self.attrs[p] = WdObject(o.label)
                    
                
    def get_triples(self, doubles: dict, prettify: bool = True, attrs: bool = False) -> list:
        """ Format triples from list of predicate * objects """
        
        format_triple = lambda triple: (
            triple if prettify else 
            ( s.iri, p.iri, o ) if attrs else 
            ( el.iri for el in triple )
        )
        
        triples = []
        for p, o in doubles.items():
            
            if isinstance(o, list):
                triples.extend([ 
                    format_triple(( self.wd_ent, p, el ))
                    for el in o 
                ])
            else:
                triple = format_triple(( self.wd_ent, p, o ))
                triples.append(triple)
                
        return triples
    
    def get_rels(self, prettify: bool = True) -> list:
        """ Return formatted relations to other entities """
        return self.get_triples(self.rels, prettify=prettify)
    
    def get_attrs(self, prettify: bool = True) -> list:
        """ Return formatted attributes """
        return self.get_triples(self.attrs, prettify=prettify, attrs=True)
    
    def get_links(self, prettify: bool = True) -> list:
        """ Return formatted links to other entities found in text """
        return self.get_triples(self.links, prettify=prettify)


In [6]:
ents = {}

for ent in text.ents:
    ent = NamedEntity(ent)
    ent.query_ent()
    ents["wd:" + ent.wid] = ent
    
ents

{'wd:Q29468': Republicans, 'wd:Q29552': Democrats, 'wd:Q22686': Donald Trump}

In [7]:
for ent in ents.values():
    ent.link_ents(ents)
    ent.stringify_ents()
    print(ent)
    print(ent.links)
    print()

Republicans
{opposite of: Democratic Party}

Democrats
{opposite of: Republican Party}

Donald Trump
{member of political party: Republican Party}



In [8]:
ent = ents['wd:Q29468']

In [15]:
rels = ent.get_rels()
rels

[(Republicans, country, United States of America),
 (Republicans, instance of, political party),
 (Republicans, founded by, Whig Party),
 (Republicans, founded by, Free Soil Party),
 (Republicans, follows, Whig Party),
 (Republicans, follows, Free Soil Party),
 (Republicans, followed by, Radical Democracy Party),
 (Republicans, headquarters location, Washington, D.C.),
 (Republicans,
  logo image,
  http://commons.wikimedia.org/wiki/Special:FilePath/GOP%20logo.svg),
 (Republicans, official website, https://www.gop.com/),
 (Republicans, opposite of, Democratic Party),
 (Republicans, color, red),
 (Republicans, member of, International Democrat Union),
 (Republicans, chairperson, Ronna Romney McDaniel),
 (Republicans, mascot, Republican Elephant),
 (Republicans,
  topic's main category,
  Category:Republican Party (United States)),
 (Republicans, political ideology, conservatism),
 (Republicans, political ideology, neoliberalism),
 (Republicans, political ideology, neoconservatism),
 (Re

In [17]:
rels[1][1].iri

'wd:P31'

In [20]:
rels[10][1].iri

'wd:P461'

In [10]:
ent.get_links()

[(Republicans, opposite of, Democratic Party)]

In [11]:
ent.get_attrs()

[(Republicans, number of seats in legislature, 52),
 (Republicans, number of seats in legislature, 211),
 (Republicans, member count, 68049840),
 (Republicans, social media followers, 2786834),
 (Republicans, short name, R),
 (Republicans, short name, GOP),
 (Republicans, Twitter username, GOP),
 (Republicans, Instagram username, gop),
 (Republicans, Giphy username, gop),
 (Republicans, country, United States of America),
 (Republicans, instance of, political party),
 (Republicans, founded by, Free Soil Party),
 (Republicans, follows, Free Soil Party),
 (Republicans, followed by, Radical Democracy Party),
 (Republicans, headquarters location, Washington, D.C.),
 (Republicans,
  logo image,
  http://commons.wikimedia.org/wiki/Special:FilePath/GOP%20logo.svg),
 (Republicans, official website, https://www.gop.com/),
 (Republicans, color, red),
 (Republicans, member of, International Democrat Union),
 (Republicans, chairperson, Ronna Romney McDaniel),
 (Republicans, mascot, Republican Elep

In [12]:
# Useless, opentapioca already spots entities
QUERY_ENTS = """
SELECT ?p ?rank 
       ( COUNT(?s) + COUNT(?o) AS ?score )

WHERE {{
  
  SERVICE wikibase:mwapi {{
      bd:serviceParam wikibase:endpoint "www.wikidata.org" ;
                      wikibase:api "EntitySearch" ;
                      wikibase:limit 10 ;
                      mwapi:search "father" ;
                      mwapi:language "en" .
      ?p wikibase:apiOutputItem mwapi:item .
      ?rank wikibase:apiOrdinal true .
  }}
  
  VALUES ?ent {ents}
  OPTIONAL {{ ?ent ?p ?o }} 
  OPTIONAL {{ ?s ?p ?ent }}
  
}}

GROUP BY ?p ?rank
ORDER BY ?score ?rank
"""